In [25]:
import json
import numpy as np
import os
os.getcwd()

'/Users/sidhartkrishnan/Documents/Programming/Projects/cs236/carafe/src'

In [26]:
RESULTS_FILE = "../results/final_results/expmt/output_store.json"
DATASET_FILE = "../data/verified_data/dataset-test.json"



with open(DATASET_FILE, "r") as f:
    dataset = json.load(f)


In [48]:
def prediction(x, context='no-context', direct_ans=False):
    if not direct_ans:
        return x[context]['pred']['prediction']
    return make_num(x[context]['metadata']['llm_output'].split("=")[0])

def answer(data):
    return data['answer']

def make_num(y):
    try:
        if type(y) in [int, float, np.float64]:
            return y
        if type(y) == str:
            return np.float64(y.split(" ")[0])
    except:
        return None

def accuracy_metric(y, y_hat, max_error=3):
    y = make_num(y)
    y_hat = make_num(y_hat)

    if y is None or y_hat is None:
        return 0
    if y < 0 or y_hat < 0:
        return 0
    if y == 0 and y_hat == 0:
        return 1
    elif y == 0 or y_hat == 0:
        return max(0, 1-np.abs(np.log10(np.abs(y - y_hat))))
    # elif y/y_hat == 0:
    #     return 0
    try:
        return max(0, max_error-np.abs(np.log10(y/y_hat)))/max_error
    except:
        return 0

In [49]:
accuracies = [accuracy_metric(data['answer'], prediction(result)) for data, result in zip(dataset, results)]
print(f"{np.mean(accuracies)} +/- {np.std(accuracies)}")

0.553329661157046 +/- 0.37337746382382725


In [50]:
def evaluate(result_file, context='no-context', direct_ans=False):
    with open(result_file, "r") as f:
        results = json.load(f)
    accuracies = []
    for data, result in zip(dataset, results):
        accuracy = accuracy_metric(data['answer'], prediction(result, context=context, direct_ans=direct_ans))
        if data['units'] == 'percents':
            accuracy = min(accuracy, accuracy_metric(data['answer']/100, prediction(result, context=context, direct_ans=direct_ans)))
        accuracies.append(accuracy)
    print(f"{np.mean(accuracies)} ")
    return accuracies

evaluate("../results/final_results/expmt3/output_store.json")
evaluate("../results/final_results/expmt1/output_store.json")
evaluate("../results/final_results/expmt0.5/output_store_k=5.json", direct_ans=True)
pass

0.5325138955648877 
0.47553926597531726 
0.5303093688482068 


In [51]:
evaluate("../results/final_results/expmt3/output_store.json", context='regular-context')
evaluate("../results/final_results/expmt1/output_store.json", context='regular-context')
evaluate("../results/final_results/expmt0/output_store_k=5.json", context='regular-context', direct_ans=True)
pass

0.8511912018790331 
0.8640622826621209 
0.7486636902277158 


# Q0: Under ideal conditions, the bacterium eNSO can reproduce in 10 minutes. eNSO is rodlike with a diameter of 1 micro meters and a length of 5 micro meters. If you start a culture of one eNSO bacterium and they only have onions as its food source, how many minutes elapse before the culture has consumed an amount equal to the onion mass consumed in the US in a year?
# R0: This is the main question we want to answer.
# Q1: What is the mass of the onion consumed in the US in a year?
# R1: We need to know the mass of the onion consumed in the US in a year because that is the amount of mass that the culture needs to consume.
# F1: The mass of the onion consumed in the US in a year is 2.3 million metric tons. We must convert this into kg by multiplying by 1e+6.
A1=1e+6 * 2.3e+6 # (kg)
# Q2: What is the mass of one eNSO bacterium?
# R2: We need to know the mass of one eNSO bacterium because that is the amount of mass that one eNSO bacterium consumes when it reproduces.
# Q3: What is the vo